In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()

roi = config.roi
data_folder = config.data_folder
_, lulc, _ = desired_lulc()

biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
proj = biomass_raw.projection().getInfo()

## Surrounding mature forests

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Mean fragmentation of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1)

In [20]:
def get_mature_mask(method, lulc):
    config = ProjectConfig()

    if method == 'eu':
        annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(config.roi)
        return annual_changes.select('Dec2020').eq(1).selfMask()
    elif method == 'mapbiomas':
        return lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask()

def get_mature_biomass(method, biomass_raw, mature_mask):
    if method == 'eu':
        return biomass_raw.updateMask(mature_mask).rename("mature_biomass")
    elif method == 'mapbiomas':
        edge_detec = mature_mask.unmask(-1).zeroCrossing()
        distance_to_edge = edge_detec.fastDistanceTransform().sqrt() \
            .multiply(ee.Image.pixelArea().sqrt()).rename("distance_to_edge")
        distance_to_edge = distance_to_edge.gt(1000).selfMask().updateMask(mature_mask)
        return biomass_raw.updateMask(distance_to_edge).rename("mature_biomass")

def export_mature_biomass(method, export = True):

    biomass_raw = ee.Image(f"{data_folder}/raw/biomass")
    proj = biomass_raw.projection().getInfo()

    mature_mask = get_mature_mask(method, lulc)
    sur_cover = mature_mask.unmask(0).focalMean(radius = 1000, units = "meters").float().rename("sur_cover")
    mature_biomass = get_mature_biomass(method, biomass_raw, mature_mask)

    if export:
        export_image(mature_biomass, "mature_biomass1", folder = method, crsTransform = proj['transform'])
        export_image(mature_biomass, "mature_biomass_5001", folder = method, scale = 500)
        export_image(sur_cover, "sur_cover1", folder = method, scale = 30)

    return mature_biomass

# export_mature_biomass('eu')
# export_mature_biomass('mapbiomas')

In [5]:
def get_mature_biomass_vector(mature_biomass, method):
    # there are time out issues with larger mature_biomass images.
    # to get around it, we reduce to vectors over the geometries of the biomes we are considering
    # and we exclude the pixels that are within the given search radius of the biome edges

    biome = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")
    amaz_atla_pant = biome.filter(ee.Filter.inList('CD_Bioma', [1, 4, 6]))
    mature_biomass = mature_biomass.clip(amaz_atla_pant)

    mature_biomass_vector = mature_biomass.sample(region = mature_biomass.geometry(), scale = 1000, geometries = True)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = mature_biomass_vector,
        assetId = f"{data_folder}/{method}/mature_biomass_vector",
        description = "mature_biomass_vector"
    )

    task.start()

mature_biomass = ee.Image(f"{data_folder}/eu/mature_biomass")
get_mature_biomass_vector(mature_biomass, 'eu')


In [10]:
def get_distance_to_nearest_mature(method):

    secondary = ee.Image(f"{data_folder}/{method}/secondary")
    secondary = secondary.select("age")

    mature_biomass_vector = ee.FeatureCollection(f"{data_folder}/{method}/mature_biomass_vector")
    # change radius depending on data resolution and patterns
    distance_to_nearest_mature = mature_biomass_vector.distance(searchRadius = 10000)\
        .toInt().updateMask(secondary)

    export_image(distance_to_nearest_mature, "distance_to_nearest_mature", \
                 folder = method, crsTransform = proj['transform'], scale = 1000)

    return distance_to_nearest_mature

distance_to_nearest_mature = get_distance_to_nearest_mature('eu')


In [12]:
def buffer_distance(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(1000)))
    return buffered

def get_distances_vector_buffer(distance_to_nearest_mature, method):
    categorical = ee.Image(f"{data_folder}/categorical")
    
    biome_mask = categorical.select('biome').eq(1) \
    .Or(categorical.select('biome').eq(4)) \
    .Or(categorical.select('biome').eq(6))

    distance_to_nearest_mature = distance_to_nearest_mature \
        .addBands(biome_mask)

    distances_vector = distance_to_nearest_mature.stratifiedSample(
        numPoints = 50000, classBand = "biome", region = roi, geometries = True
    )
    
    distances_vector_buffer = distances_vector.map(buffer_distance)

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = distances_vector_buffer,
        assetId = f"{data_folder}/{method}/distances_vector_buffer",
        description = "distances_vector_buffer"
    )

    task.start()

distance_to_nearest_mature = ee.Image(f"{data_folder}/eu/distance_to_nearest_mature")
get_distances_vector_buffer(distance_to_nearest_mature, 'eu')

In [6]:
# mature_biomass = ee.Image(f"{data_folder}/eu/mature_biomass")
# mature_biomass_vector = ee.FeatureCollection(f"{data_folder}/eu/mature_biomass_vector")
# biome = ee.FeatureCollection(f"{data_folder}/raw/biomes_br")
# amaz_atla_pant = biome.filter(ee.Filter.inList('CD_Bioma', [1, 4, 6]))
# annual_changes = ee.ImageCollection('projects/JRC/TMF/v1_2023/AnnualChanges').mosaic().clip(config.roi)
# nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')
# basemap3 = nicfi.filter(ee.Filter.date('2020-07-01', '2020-12-01')).first()
# distance_to_nearest_mature = ee.Image(f"{data_folder}/eu/distance_to_nearest_mature")
# vis_planet = {'bands': ['R', 'G', 'B'], 'min': 64, 'max': 5454, 'gamma': 1.8}

# map = geemap.Map()
# map.addLayer(amaz_atla_pant, {}, 'amaz_atla_pant')
# map.addLayer(annual_changes, {}, 'annual_changes')
# map.addLayer(basemap3, vis_planet, 'basemap3')
# map.addLayer(distance_to_nearest_mature, {'min': 0, 'max': 100000, 'palette': ['blue', 'red']}, 'distance_to_nearest_mature')
# map.addLayer(mature_biomass, {'min': 0, 'max': 200, 'palette': ['yellow', 'green']}, 'mature_biomass')
# map.addLayer(mature_biomass_vector, {}, 'mature_biomass_vector')
# map

In [4]:
def export_nearest_mature_biomass(distances_vector_buffer, mature_biomass, method):

    nearest_mature_biomass = mature_biomass.reduceRegions(distances_vector_buffer, ee.Reducer.firstNonNull(), 
                                            crs = proj['crs'], crsTransform = proj['transform'])

    # Export the mean_biomass_image as an asset
    task = ee.batch.Export.table.toAsset(
        collection = nearest_mature_biomass,
        assetId = f"{data_folder}/{method}/nearest_mature_biomass",
        description = "nearest_mature_biomass"
    )

    task.start()

mature_biomass = ee.Image(f"{data_folder}/eu/mature_biomass_500")
distances_vector_buffer = ee.FeatureCollection(f"{data_folder}/eu/distances_vector_buffer")
export_nearest_mature_biomass(distances_vector_buffer, mature_biomass, 'eu')

In [7]:
def export_nearest_mature_biomass_image(nearest_mature_biomass, method):

    # change geometries from polygons to points
    nearest_mature_biomass = nearest_mature_biomass.map(lambda f: ee.Feature(f.geometry().centroid()).copyProperties(f))

    # Convert the FeatureCollection to an image
    nearest_mature_biomass_image = nearest_mature_biomass.reduceToImage(
        properties = ['first'],
        reducer = ee.Reducer.first()
    ).toInt16().rename('nearest_mature_biomass')

    export_image(nearest_mature_biomass_image, "nearest_mature_biomass_image", \
                 folder = method, crsTransform = proj['transform'])

nearest_mature_biomass = ee.FeatureCollection(f"{data_folder}/eu/nearest_mature_biomass")
export_nearest_mature_biomass_image(nearest_mature_biomass, 'eu')


In [ ]:
gaus_kernel = ee.Kernel.gaussian(radius = 1000, sigma = 200, units = "meters")
mat_gaus_ker = (
    mature_biomass.focalMean(
        radius = 1000, units = "meters", kernel = gaus_kernel, iterations = 1
    )
    .rename("mat_biomass")
    .float()
)

# the fragmentation data also does not cover the entire landscape - it is targeted towards mature forests.
# frag file from Ma et al was originally reprojected in QGIS into WGS84 (original projection is not compatible with google earth engine)
# frag values vary from 0 to 1
frag = ee.Image(f"{data_folder}/raw/frag").rename("frag").clip(age.geometry())
frag_ker = frag.focalMean(radius = 5000, units = "meters").rename("frag")

# export_image(mat_gaus_ker, "mat_gaus_ker", scale = 30)
# export_image(frag_ker, "frag_ker", scale = 30)
